
[![ebac_logo-data_science.png](https://raw.githubusercontent.com/diegobezzi/DataScience_Estudos/main/ebac-course-utils/media/logo/ebac_logo-data_science.png)](https://github.com//diegobezzi/DataScience_Estudos)
<!-- <img src="https://raw.githubusercontent.com/diegobezzi/DataScience_Estudos/main/ebac-course-utils/media/logo/ebac_logo-data_science.png" alt="ebac_logo-data_science"> -->

---

<!-- # **Profissão: Cientista de Dados** -->
### **Módulo 23** | Combinação de modelos I | Exercício 1

**Aluno:** [Diego Perez Bezzi Vilas Boas](https://www.linkedin.com/in/diegobezzi/)<br>
**Data:** 12 de junho de 2024.

---

# Tarefa 02

**1.** Monte um passo a passo para o algoritmo RF

> Semelhante ao *Bagging*, o ***Random Forest*** consiste nos seguintes passos:
> 1. ***Bootstrap + Feature Selection:*** Assim como no *Bagging*, o *Random Forest* utiliza amostras aleatórias com reposição do conjunto de dados de treinamento original. No entanto, em cada uma dessas amostras, apenas um subconjunto aleatório de variáveis é selecionado (*feature selection*). No caso de problemas de classificação, geralmente é recomendado escolher a raiz quadrada do número total de variáveis, enquanto em problemas de regressão, um terço das variáveis é comumente usado.
> 2. **Modelagem com árvores de decisão:** Nesta etapa, um modelo de *Machine Learning*, especificamente uma árvore de decisão, é treinado de forma independente em cada amostra *bootstrap* com as variáveis aleatórias que foram definidas no passo anterior.
> 3. **Agregação:** Por fim, os resultados de cada modelo independente (cada árvore de decisão) são agregados para obter uma previsão final. Em problemas de classificação, a agregação é geralmente feita por meio do voto majoritário, em que a classe prevista com mais frequência pelos modelos individuais é selecionada como a classe final. Já em problemas de regressão, a agregação é realizada calculando a média das previsões dos modelos individuais.

**2.** Explique com suas palavras o Random Forest

> ***Random Forest*** pode ser definido como uma extensão do *Bootstrap Aggregating*, conhecido como *Bagging*, que oferece melhorias em desempenho e resultados. Assim como no *Bagging*, o *Random Forest* é um método de combinação de modelos de *Machine Learning*, em que cada modelo é treinado em variações do conjunto de dados original. Essas variações, chamadas de *amostras bootstrap*, consistem em subconjuntos aleatórios do conjunto de dados original, com possíveis repetições, mantendo o mesmo número de linhas. No entanto, o *Random Forest* diferencia-se ao utilizar apenas uma quantidade determinada e aleatória de variáveis em cada modelo, especificamente em modelos de árvore de decisão. Essa abordagem tem o objetivo de reduzir a variância dos resultados e atenuar o risco de *overfitting*. Para obter a previsão final, a agregação dos modelos é realizada de forma semelhante ao *Bagging*, onde a média é utilizada para regressão e votação para classificação.

**3.** Qual a diferença entre Bagging e Random Forest?

> A diferença entre *Bagging* e *Random Forest* está no desempenho e na redução da variância entre os resultados dos modelos. *Bagging* é uma técnica mais geral de combinação de modelos de *Machine Learning* utilizando amostragem aleatória, enquanto *Random Forest* é uma variação mais específica desse método que utiliza árvores de decisão. No *Random Forest*, cada modelo é treinado em subconjuntos aleatórios do conjunto de dados original, com uma quantidade reduzida e aleatória de variáveis consideradas em cada modelo. Essa abordagem aumenta a robustez do modelo e, ao combinar os resultados dos modelos, a previsão final tende a ser mais precisa do que no *Bagging*. Portanto, *Random Forest* é uma extensão aprimorada do *Bagging*, fornecendo um desempenho melhorado e uma redução adicional na variância dos resultados.
>
>> **"*Random Forest* funciona melhor que o *Bagging*, pois as árvores amostradas são mais independentes (menor correlação)."**

**4.** (Opcional) Implementar em python o Random Forest
> - Bootstrap
> - Feature selection
> - Modelagem com Decision trees
> - Agregação

In [4]:
# Import das bibliotecas:

import numpy  as np
import pandas as pd

from sklearn.datasets        import load_wine
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [5]:
# Exemplo da técnica Random Forest para problemas de classificação:

X = load_wine().data
y = load_wine().target

df = pd.DataFrame(X, columns=load_wine().feature_names)
df['target'] = y

def rf_classifier(df:pd.DataFrame, 
                  num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                  test_size:float=0.25
                 ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(np.sqrt(X_train.shape[1])),  # Cálculo da raiz quadrada da quantidade de variáveis
                                 axis=1)
        
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

rf_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.9830508474576272


,y_test,y_pred
0,2,2
1,1,1
2,1,1
3,1,1
4,0,0
5,0,0
6,1,1
7,2,2
8,1,1
9,1,1


In [6]:
# Exemplo da técnica Random Forest para problemas de regressão:

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def rf_regressor(df:pd.DataFrame, 
                 num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                 test_size:float=0.25
                ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(X_train.shape[1]/3),  # Cálculo da quantidade de variáveis dividida por 3
                                 axis=1)
        
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
rf_regressor(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 4134.850916894978
Coefficient of determination: 0.29423659026602156


,y_test,y_pred
0,141.0,138.7
1,69.0,142.4
2,88.0,174.0
3,65.0,65.9
4,261.0,239.0
...,...,...
141,185.0,213.0
142,158.0,131.7
143,258.0,103.3
144,53.0,141.2


---